In [1]:
import pandas as pd
import numpy as np
import os
import joblib as joblib
import pickle

from tqdm.auto import tqdm
from datetime import datetime
from scipy.stats import zscore

# from ReMASTER.system import get_data_dir

tqdm.pandas()

/Users/areejmirani/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ_5Years_8_11_2024.csv'

# Initialize an empty DataFrame for the combined data
# data = pd.DataFrame()
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to inspect its structure
data.head()

,Time,Open,High,Low,Close,Volume
0,8/11/2019 23:05,9073.25,9098.50,9073.00,9092.50,1758
1,8/11/2019 23:10,9093.25,9095.50,9089.75,9092.75,438
2,8/11/2019 23:15,9093.00,9096.25,9088.00,9089.75,590
3,8/11/2019 23:20,9090.25,9090.25,9086.00,9087.00,278
4,8/11/2019 23:25,9086.75,9088.25,9079.75,9083.75,711


## updated method to ensure this set fits the csi300 form with alpha158 form etc.

In [3]:
# import pandas as pd
# import numpy as np
# from datetime import datetime

# # 1. Convert 'Date' to datetime format
# data['Date'] = pd.to_datetime(data['Time'])

# # Set 'Date' as the index for time-based operations
# data.set_index('Date', inplace=True)

# # 2. Define the date ranges for the new splits (2020-2024)
# train_end = datetime(2022, 3, 31)  # Q1 2022 ends
# valid_start = datetime(2022, 4, 1)  # Q2 2022 starts
# valid_end = datetime(2022, 6, 30)  # Q2 2022 ends
# test_start = datetime(2022, 7, 1)  # Q3 2022 starts
# test_end = datetime(2024, 12, 31)  # Q4 2024 ends
# # 
# # 3. Create train, validation, and test sets based on the new splits
# train_data = data[data.index <= train_end]
# valid_data = data[(data.index >= valid_start) & (data.index <= valid_end)]
# test_data = data[(data.index >= test_start) & (data.index <= test_end)]

# # 4. Feature Engineering - Add Alpha158-like features
# def calculate_alpha158_features(df):
#     df['close_open'] = (df['Close'] - df['Open']) / df['Open']  # Close to Open ratio
#     df['high_low'] = (df['High'] - df['Low']) / df['Open']  # High-Low to Open ratio
#     df['close_open_highlow'] = (df['Close'] - df['Open']) / (df['High'] - df['Low'] + 1e-12)  # Close-Open to High-Low ratio
#     df['high_max_open_close'] = (df['High'] - np.maximum(df['Open'], df['Close'])) / df['Open']  # High minus max(Open, Close) to Open ratio
#     df['volume_change'] = df['Volume'].pct_change()  # Percentage change in volume
#     return df

# # Apply the feature engineering to each dataset
# train_data = calculate_alpha158_features(train_data)
# valid_data = calculate_alpha158_features(valid_data)
# test_data = calculate_alpha158_features(test_data)

# # 5. Lookback Window and Prediction Interval
# lookback_window = 8  # 8 days lookback window
# prediction_interval = 5  # 5 days prediction interval

# def create_lookback_features(df, lookback_window, prediction_interval):
#     df['ma_close'] = df['Close'].rolling(window=lookback_window).mean()  # Moving average of the closing price
#     df['ma_volume'] = df['Volume'].rolling(window=lookback_window).mean()  # Moving average of volume
#     df['price_change'] = df['Close'].pct_change(periods=prediction_interval)  # Percentage change in closing price over the prediction interval
#     df['volatility'] = df['Close'].rolling(window=lookback_window).std()  # Standard deviation (volatility) over the lookback window
#     return df

# # Apply the lookback features to each dataset
# train_data = create_lookback_features(train_data, lookback_window, prediction_interval)
# valid_data = create_lookback_features(valid_data, lookback_window, prediction_interval)
# test_data = create_lookback_features(test_data, lookback_window, prediction_interval)

# # 6. Optional: Handle missing data (NaN) in the feature columns
# train_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
# valid_data.fillna(method='ffill', inplace=True)
# test_data.fillna(method='ffill', inplace=True)

# # 7. Optional: Scaling the features
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# features = ['close_open', 'high_low', 'close_open_highlow', 'high_max_open_close', 'volume_change', 'ma_close', 'ma_volume', 'price_change', 'volatility']
# train_data[features] = scaler.fit_transform(train_data[features])
# valid_data[features] = scaler.transform(valid_data[features])
# test_data[features] = scaler.transform(test_data[features])

# # 8. Save the processed datasets
# train_data.to_csv('train_data_2020_2022.csv')
# valid_data.to_csv('valid_data_2022.csv')
# test_data.to_csv('test_data_2022_2024.csv')

# # 9. Show the final processed data
# print("Training Set:")
# print(train_data.head())
# # print("Validation Set:")
# # print(valid_data.head())
# # print("Test Set:")
# # print(test_data.head())

# #10. check column names
# print("training set: ")
# print(train_data.columns)
# # print("validation set: ")
# # print(valid_data.columns)
# # print("test set: ")
# # print(test_data.columns)

# #10. check datatypes
# print("training set: ")
# print(train_data.dtypes)
# # print("validation set: ")
# # print(valid_data.dtypes)
# # print("test set: ")
# # print(test_data.dtypes)

# #10. check shape (len, width) 
# print("training set: ")
# print(train_data.shape)
# # print("validation set: ")
# # print(valid_data.shape)
# # print("test set: ")
# # print(test_data.shape)



# reformat to fit multi-index and get the specified features that csi300 has

this is what csi300's formulas were:

MultiIndex([('feature',                              '($close-$open)/$open'),
            ('feature',                                '($high-$low)/$open'),
            ('feature',                 '($close-$open)/($high-$low+1e-12)'),
            ('feature',              '($high-Greater($open, $close))/$open'),
            ('feature', '($high-Greater($open, $close))/($high-$low+1e-12)'),
            ('feature',                  '(Less($open, $close)-$low)/$open'),
            ('feature',     '(Less($open, $close)-$low)/($high-$low+1e-12)'),
            ('feature',                       '(2*$close-$high-$low)/$open'),
            ('feature',          '(2*$close-$high-$low)/($high-$low+1e-12)'),
            ('feature',                                      '$open/$close'),
            ...
            ('feature',          'Mask(Std($amount,20)/$amount,'SH000906')'),
            ('feature',  'Mask(Mean($close/Ref($close,1)-1,30),'SH000906')'),
            ('feature',   'Mask(Std($close/Ref($close,1)-1,30),'SH000906')'),
            ('feature',         'Mask(Mean($amount,30)/$amount,'SH000906')'),
            ('feature',          'Mask(Std($amount,30)/$amount,'SH000906')'),
            ('feature',  'Mask(Mean($close/Ref($close,1)-1,60),'SH000906')'),
            ('feature',   'Mask(Std($close/Ref($close,1)-1,60),'SH000906')'),
            ('feature',         'Mask(Mean($amount,60)/$amount,'SH000906')'),
            ('feature',          'Mask(Std($amount,60)/$amount,'SH000906')'),
            (  'label',             'Ref($close, -5) / Ref($close, -1) - 1')],
           length=222)

In [4]:
# import pandas as pd
# import numpy as np
# from datetime import datetime
# from sklearn.preprocessing import StandardScaler

# # 1. Convert 'Date' to datetime format
# data['Date'] = pd.to_datetime(data['Time'])

# # Set 'Date' as the index for time-based operations
# data.set_index('Date', inplace=True)

# # 2. Define the date ranges for the new splits (2020-2024)
# train_end = datetime(2022, 3, 31)  # Q1 2022 ends
# valid_start = datetime(2022, 4, 1)  # Q2 2022 starts
# valid_end = datetime(2022, 6, 30)  # Q2 2022 ends
# test_start = datetime(2022, 7, 1)  # Q3 2022 starts
# test_end = datetime(2024, 12, 31)  # Q4 2024 ends

# # 3. Create train, validation, and test sets based on the new splits
# train_data = data[data.index <= train_end]
# valid_data = data[(data.index >= valid_start) & (data.index <= valid_end)]
# test_data = data[(data.index >= test_start) & (data.index <= test_end)]

# # 4. Feature Engineering - Add Alpha158-like features (more extensive)
# def calculate_alpha158_features(df):
#     # Basic features
#     df['close_open'] = (df['Close'] - df['Open']) / df['Open']  # Close to Open ratio
#     df['high_low'] = (df['High'] - df['Low']) / df['Open']  # High-Low to Open ratio
#     df['close_open_highlow'] = (df['Close'] - df['Open']) / (df['High'] - df['Low'] + 1e-12)  # Close-Open to High-Low ratio
#     df['high_max_open_close'] = (df['High'] - np.maximum(df['Open'], df['Close'])) / df['Open']  # High minus max(Open, Close) to Open ratio
#     df['volume_change'] = df['Volume'].pct_change()  # Percentage change in volume
    
#     # Additional advanced features (based on CSI300)
#     df['high_minus_low'] = df['High'] - df['Low']  # High-Low difference
#     df['low_max_open_close'] = df['Low'] - np.maximum(df['Open'], df['Close'])  # Low minus max(Open, Close)
#     df['high_to_open'] = (df['High'] - df['Open']) / df['Open']  # High to Open ratio
#     df['low_to_open'] = (df['Low'] - df['Open']) / df['Open']  # Low to Open ratio
#     df['close_to_high'] = (df['Close'] - df['High']) / df['High']  # Close to High ratio
#     df['close_to_low'] = (df['Close'] - df['Low']) / df['Low']  # Close to Low ratio

#     # Rolling features (Moving Averages and Volatility)
#     df['ma_close'] = df['Close'].rolling(window=8).mean()  # 8-day moving average of Close
#     df['ma_volume'] = df['Volume'].rolling(window=8).mean()  # 8-day moving average of Volume
#     df['volatility'] = df['Close'].rolling(window=8).std()  # 8-day rolling standard deviation (volatility)
#     df['price_change'] = df['Close'].pct_change(periods=5)  # 5-day percentage change in Close

#     # More moving averages (longer periods)
#     df['ma_30_close'] = df['Close'].rolling(window=30).mean()  # 30-day moving average of Close
#     df['ma_60_close'] = df['Close'].rolling(window=60).mean()  # 60-day moving average of Close
#     df['ma_30_volume'] = df['Volume'].rolling(window=30).mean()  # 30-day moving average of Volume

#     # Adding additional feature like a ratio involving rolling standard deviation
#     df['std_close'] = df['Close'].rolling(window=30).std()  # 30-day rolling standard deviation
#     df['std_volume'] = df['Volume'].rolling(window=30).std()  # 30-day rolling volume standard deviation

#     return df

# # Apply the feature engineering to each dataset
# train_data = calculate_alpha158_features(train_data)
# valid_data = calculate_alpha158_features(valid_data)
# test_data = calculate_alpha158_features(test_data)

# # 5. Lookback Window and Prediction Interval (for future prediction, if needed)
# lookback_window = 8  # 8 days lookback window
# prediction_interval = 5  # 5 days prediction interval

# def create_lookback_features(df, lookback_window, prediction_interval):
#     df['ma_close'] = df['Close'].rolling(window=lookback_window).mean()  # Moving average of the closing price
#     df['ma_volume'] = df['Volume'].rolling(window=lookback_window).mean()  # Moving average of volume
#     df['price_change'] = df['Close'].pct_change(periods=prediction_interval)  # Percentage change in closing price over the prediction interval
#     df['volatility'] = df['Close'].rolling(window=lookback_window).std()  # Standard deviation (volatility) over the lookback window
#     return df

# # Apply the lookback features to each dataset
# train_data = create_lookback_features(train_data, lookback_window, prediction_interval)
# valid_data = create_lookback_features(valid_data, lookback_window, prediction_interval)
# test_data = create_lookback_features(test_data, lookback_window, prediction_interval)

# # 6. Optional: Handle missing data (NaN) in the feature columns
# train_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
# valid_data.fillna(method='ffill', inplace=True)
# test_data.fillna(method='ffill', inplace=True)

# # 7. Optional: Scaling the features
# scaler = StandardScaler()
# features = ['close_open', 'high_low', 'close_open_highlow', 'high_max_open_close', 'volume_change', 'ma_close', 'ma_volume', 'price_change', 'volatility']
# train_data[features] = scaler.fit_transform(train_data[features])
# valid_data[features] = scaler.transform(valid_data[features])
# test_data[features] = scaler.transform(test_data[features])

# # 8. Feature Engineering - Adding the MultiIndex format as you requested
# def add_multiindex_format(df):
#     # Recompute the features in the format you provided
#     features = ['close_open', 'high_low', 'close_open_highlow', 'high_max_open_close', 'high_max_open_close_highlow',
#                 'high_minus_low', 'low_max_open_close', 'high_to_open', 'low_to_open', 'close_to_high', 'close_to_low',
#                 'ma_close', 'ma_volume', 'volatility', 'price_change', 'ma_30_close', 'ma_60_close', 'ma_30_volume',
#                 'std_close', 'std_volume']

#     df_features = df[features]

#     # Now, we'll change the column format to MultiIndex
#     columns = pd.MultiIndex.from_tuples([('feature', col) for col in df_features.columns])

#     # Assign this MultiIndex to the dataframe
#     df_features.columns = columns

#     return df_features

# # Apply the MultiIndex formatting to the datasets
# train_data = add_multiindex_format(train_data)
# valid_data = add_multiindex_format(valid_data)
# test_data = add_multiindex_format(test_data)

# # 9. Save the processed datasets (after formatting)
# train_data.to_csv('train_data_2020_2022.csv')
# valid_data.to_csv('valid_data_2022.csv')
# test_data.to_csv('test_data_2022_2024.csv')

# # 10. Show the final processed data
# print("Training Set:")
# print(train_data.head())

# # 11. Check column names and datatypes
# print("Training set columns: ")
# print(train_data.columns)
# print("Training set datatypes: ")
# print(train_data.dtypes)

# # 12. Check shape (len, width)
# print("Training set shape: ")
# print(train_data.shape)


# make it multi-index for features and labels like the csi300 set

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from scipy.stats import linregress

# 1. Convert 'Date' to datetime format
data['Date'] = pd.to_datetime(data['Time'])

# Set 'Date' as the index for time-based operations
data.set_index('Date', inplace=True)

# 2. Define the date ranges for the new splits (2020-2024)
train_end = datetime(2022, 3, 31)  # Q1 2022 ends
valid_start = datetime(2022, 4, 1)  # Q2 2022 starts
valid_end = datetime(2022, 6, 30)  # Q2 2022 ends
test_start = datetime(2022, 7, 1)  # Q3 2022 starts
test_end = datetime(2024, 12, 31)  # Q4 2024 ends

# 3. Create train, validation, and test sets based on the new splits
train_data = data[data.index <= train_end]
valid_data = data[(data.index >= valid_start) & (data.index <= valid_end)]
test_data = data[(data.index >= test_start) & (data.index <= test_end)]

# 4. Feature Engineering - Add CSI300-like features. CHOOSE 22 FEATURES
def calculate_csi300_features(df):
    # Basic features matching CSI300 format

    # Price Change Ratios
    df['(close-open)/open'] = (df['Close'] - df['Open']) / df['Open']
    df['(high-low)/open'] = (df['High'] - df['Low']) / df['Open']
    df['(2*close-high-low)/open'] = (2 * df['Close'] - df['High'] - df['Low']) / df['Open']
    print("Price Change Ratios done")
    
    # Moving Averages and Standard Deviations (Volatility)
    df['Mean(close,5)/close'] = df['Close'].rolling(window=5).mean() / df['Close']
    df['Mean(close,10)/close'] = df['Close'].rolling(window=10).mean() / df['Close']
    df['Mean(close,20)/close'] = df['Close'].rolling(window=20).mean() / df['Close']
    df['Std(close,5)/close'] = df['Close'].rolling(window=5).std() / df['Close']
    df['Std(close,10)/close'] = df['Close'].rolling(window=10).std() / df['Close']
    df['Std(close,20)/close'] = df['Close'].rolling(window=20).std() / df['Close']
    print("Moving Averages and Standard Deviations done")

    # Relative Strength and Trends (Slope calculation)
    for window in [5, 10, 20]:
        df[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
            lambda x: linregress(range(len(x)), x).slope / x[-1] if len(x) == window else np.nan
        )
    print("Relative Strength and Trends done")

    # Volume-Weighted Measures
    df['VWAP/close'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum() / df['Close']
    df['Mean(volume,5)/(volume+1e-12)'] = df['Volume'].rolling(window=5).mean() / (df['Volume'] + 1e-12)
    df['Std(volume,5)/(volume+1e-12)'] = df['Volume'].rolling(window=5).std() / (df['Volume'] + 1e-12)
    print("Volume-Weighted Measures done")

    # Historical Close Ratios
    df['Ref(close,5)/close'] = df['Close'].shift(5) / df['Close']
    df['Ref(close,10)/close'] = df['Close'].shift(10) / df['Close']
    df['Ref(close,20)/close'] = df['Close'].shift(20) / df['Close']
    print("Historical Close Ratios done")

    # Correlation Measures
    df['Corr(close,log(volume+1),5)'] = df['Close'].rolling(window=5).corr(np.log(df['Volume'] + 1))
    df['Corr(close/Ref(close,1),log(volume/Ref(volume,1)+1),5)'] = (
        (df['Close'] / df['Close'].shift(1)).rolling(window=5)
        .corr(np.log(df['Volume'] / df['Volume'].shift(1) + 1))
    )
    print("Correlation Measures done")

    # Price Extremes and Ratios
    df['Max(high,5)/close'] = df['High'].rolling(window=5).max() / df['Close']
    df['Min(low,5)/close'] = df['Low'].rolling(window=5).min() / df['Close']
    print("Price Extremes and Ratios done")

    # 6. Add target label (future reference)
    df['label'] = df['Close'].shift(-5) / df['Close'].shift(-1) - 1  # Ref($close, -5) / Ref($close, -1) - 1
    
    return df

# Apply the feature engineering to each dataset
train_data = calculate_csi300_features(train_data)
valid_data = calculate_csi300_features(valid_data)
test_data = calculate_csi300_features(test_data)

# 7. Optional: Handle missing data (NaN) in the feature columns
train_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
valid_data.fillna(method='ffill', inplace=True)
test_data.fillna(method='ffill', inplace=True)

# 8. Format the dataframe to MultiIndex
def format_to_multiindex(df):
    # Define the MultiIndex for features
    columns = pd.MultiIndex.from_tuples([('feature', col) for col in df.columns if col != 'label'] + [('label', 'Ref($close, -5) / Ref($close, -1) - 1')])
    # Assign this MultiIndex to the dataframe
    df.columns = columns
    return df

# Apply MultiIndex formatting
train_data = format_to_multiindex(train_data)
valid_data = format_to_multiindex(valid_data)
test_data = format_to_multiindex(test_data)

# 9. Save the processed datasets
train_data.to_csv('train_data_2020_2022_NQ100.csv')
valid_data.to_csv('valid_data_2022_NQ100.csv')
test_data.to_csv('test_data_2022_2024_NQ100.csv')

# 10. Show the final processed data
print("Training Set:")
print(train_data.head())

# 11. Check column names and datatypes
print("Training set columns: ")
print(train_data.columns)
print("Training set datatypes: ")
print(train_data.dtypes)

# 12. Check shape (len, width)
print("Training set shape: ")
print(train_data.shape)


/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['(close-open)/open'] = (df['Close'] - df['Open']) / df['Open']
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['(high-low)/open'] = (df['High'] - df['Low']) / df['Open']
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:32: SettingWithCopyWarning: 
A value is trying 

Price Change Ratios done
Moving Averages and Standard Deviations done


/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A 

Relative Strength and Trends done
Volume-Weighted Measures done
Historical Close Ratios done
Correlation Measures done
Price Extremes and Ratios done
Price Change Ratios done
Moving Averages and Standard Deviations done


/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A 

Relative Strength and Trends done
Volume-Weighted Measures done
Historical Close Ratios done
Correlation Measures done
Price Extremes and Ratios done
Price Change Ratios done
Moving Averages and Standard Deviations done


/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:46: SettingWithCopyWarning: 
A 

Relative Strength and Trends done
Volume-Weighted Measures done
Historical Close Ratios done
Correlation Measures done
Price Extremes and Ratios done


/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.fillna(method='ffill', inplace=True)  # Forward fill missing values
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data.fillna(method='ffill', inplace=True)
/var/folders/xr/hlsgwx1x76v65d6rn3zzw6vc0000gn/T/ipykernel_20704/2770355516.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

Training Set:
                             feature                                      \
                                Time     Open     High      Low    Close   
Date                                                                       
2019-08-11 23:05:00  8/11/2019 23:05  9073.25  9098.50  9073.00  9092.50   
2019-08-11 23:10:00  8/11/2019 23:10  9093.25  9095.50  9089.75  9092.75   
2019-08-11 23:15:00  8/11/2019 23:15  9093.00  9096.25  9088.00  9089.75   
2019-08-11 23:20:00  8/11/2019 23:20  9090.25  9090.25  9086.00  9087.00   
2019-08-11 23:25:00  8/11/2019 23:25  9086.75  9088.25  9079.75  9083.75   

                                                              \
                    Volume (close-open)/open (high-low)/open   
Date                                                           
2019-08-11 23:05:00   1758          0.002122        0.002810   
2019-08-11 23:10:00    438         -0.000055        0.000632   
2019-08-11 23:15:00    590         -0.000357        0.000

# convert to .pkl files

In [6]:
# 6. Save each set as .pkl files
train_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_train.pkl'
valid_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_valid.pkl'
test_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_test.pkl'

# Save train data
with open(train_path, 'wb') as f:
    pickle.dump(train_data, f)

# Save validation data
with open(valid_path, 'wb') as f:
    pickle.dump(valid_data, f)

# Save test data
with open(test_path, 'wb') as f:
    pickle.dump(test_data, f)

# Return the paths (optional)
train_path, valid_path, test_path

('/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_train.pkl',
 '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_valid.pkl',
 '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_test.pkl')

## ensure pkl files have content 

In [7]:
# Function to load and test if data exists in each .pkl file
def test_data_in_pkl(file_path):
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        # Check if the DataFrame is not empty
        if isinstance(data, pd.DataFrame) and not data.empty:
            print(f"Data exists in {file_path} and has {len(data)} rows and {len(data.columns)} columns.")
        else:
            print(f"File {file_path} is either empty or not a DataFrame.")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Test each .pkl file
test_data_in_pkl(train_path)
test_data_in_pkl(valid_path)
test_data_in_pkl(test_path)



Data exists in /Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_train.pkl and has 170738 rows and 29 columns.
Data exists in /Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_valid.pkl and has 15723 rows and 29 columns.
Data exists in /Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_test.pkl and has 142469 rows and 29 columns.


# -

## old implementation

In [8]:
# # Step 1: Convert 'Time' column to datetime format
# data['Time'] = pd.to_datetime(data['Time'], format='%m/%d/%Y %H:%M')

# # Step 2: Check for any NA values and drop columns with NA values (if any)
# data.dropna(axis=1, inplace=True)

# # Step 3: Perform robust daily Z-score normalization on each feature dimension
# # Extract date component for daily grouping
# data['Date'] = data['Time'].dt.date
# # Group by 'Date' and normalize 'Open', 'High', 'Low', 'Close', and 'Volume' columns
# feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
# data[feature_columns] = data.groupby('Date')[feature_columns].transform(zscore)

# # Step 4: Drop 5% of the most extreme values from the 'Close' column to reduce label outliers
# # Identify upper and lower 2.5% quantiles for 'Close'
# q_low, q_high = data['Close'].quantile([0.025, 0.975])
# data = data[(data['Close'] >= q_low) & (data['Close'] <= q_high)]

# # Step 5: Drop NA rows (if any remain after filtering) and reset index
# data.dropna(inplace=True)
# data.reset_index(drop=True, inplace=True)

# # Step 6: Save the cleaned data to a new CSV file
# # Create the directory if it doesn't exist
# save_path = 'new_data/reshaped_data.csv'
# os.makedirs(os.path.dirname(save_path), exist_ok=True)
# data.to_csv(save_path, index=False)

# # Display first few rows of the cleaned data to confirm
# data.head()

# print(data.columns)


Split data into train, valid, and test sets by datetime. Earlier years are in training, recent years in test. 

In [9]:
# import pandas as pd
# import pickle
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np

# # Assuming 'data' is your original dataframe with columns like 'Date', 'Time', 'Open', 'High', 'Low', 'Close', etc.

# # 1. Combine Date and Time into a single 'datetime' column
# # Ensure that 'Date' is a string or datetime type and 'Time' is a string (e.g., "HH:MM:SS")

# # Convert 'Date' to datetime (if not already in datetime format)
# data['Date'] = pd.to_datetime(data['Date'])

# # Check the type of 'Time' column to handle it accordingly
# if data['Time'].dtype == 'datetime64[ns]':  # If Time is already in datetime64[ns]
#     data['Time'] = data['Time'].dt.time  # Extract the time part only
# else:  # If Time is a string like 'HH:MM:SS'
#     data['Time'] = pd.to_timedelta(data['Time'], errors='coerce')

# # Combine 'Date' and 'Time' into a single 'datetime' column
# data['datetime'] = data['Date'] + pd.to_timedelta(data['Time'].astype(str))

# # Drop the original 'Date' and 'Time' columns if no longer needed
# data.drop(columns=['Date', 'Time'], inplace=True)

# # 2. Feature Engineering
# data['close_open'] = (data['Close'] - data['Open']) / data['Open']
# data['high_low'] = (data['High'] - data['Low']) / data['Open']
# data['close_open_highlow'] = (data['Close'] - data['Open']) / (data['High'] - data['Low'] + 1e-12)
# data['high_max_open_close'] = (data['High'] - data[['Open', 'Close']].max(axis=1)) / data['Open']

# # 3. Handle NaN or infinite values
# # Replace NaN with the column mean, or you can use other strategies
# data.fillna(data.mean(), inplace=True)

# # Replace any infinity values with a large number or NaN
# data.replace([np.inf, -np.inf], np.nan, inplace=True)

# # Reapply fillna to handle any NaNs created by infinities
# data.fillna(data.mean(), inplace=True)

# # 4. Normalize the newly created features
# scaler = MinMaxScaler()

# # Select columns for scaling
# features = ['close_open', 'high_low', 'close_open_highlow', 'high_max_open_close']
# data[features] = scaler.fit_transform(data[features])

# # 5. Split the data into train, valid, and test
# train_size = int(0.7 * len(data))  # 70% for training
# valid_size = int(0.15 * len(data))  # 15% for validation
# test_size = len(data) - train_size - valid_size  # 15% for testing

# train_data = data[:train_size]
# valid_data = data[train_size:train_size + valid_size]
# test_data = data[train_size + valid_size:]

# # 6. Save each set as .pkl files
# train_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_train.pkl'
# valid_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_valid.pkl'
# test_path = '/Users/areejmirani/Desktop/ReMASTER/data/NQ100/NQ100_dl_test.pkl'

# # Save train data
# with open(train_path, 'wb') as f:
#     pickle.dump(train_data, f)

# # Save validation data
# with open(valid_path, 'wb') as f:
#     pickle.dump(valid_data, f)

# # Save test data
# with open(test_path, 'wb') as f:
#     pickle.dump(test_data, f)

# # Return the paths (optional)
# train_path, valid_path, test_path


Ensure data is in the .pkl files

In [10]:
# # Function to load and test if data exists in each .pkl file
# def test_data_in_pkl(file_path):
#     try:
#         with open(file_path, 'rb') as f:
#             data = pickle.load(f)
#         # Check if the DataFrame is not empty
#         if isinstance(data, pd.DataFrame) and not data.empty:
#             print(f"Data exists in {file_path} and has {len(data)} rows and {len(data.columns)} columns.")
#         else:
#             print(f"File {file_path} is either empty or not a DataFrame.")
#     except Exception as e:
#         print(f"Error reading {file_path}: {e}")

# # Test each .pkl file
# test_data_in_pkl(train_path)
# test_data_in_pkl(valid_path)
# test_data_in_pkl(test_path)